#### The goal of this notebook is to scrape information on all the mutual funds listed on the website https://www.avanza.se/fonder/lista.html. The information for each mutual fund is stored in its own webpage, so we need to navigate through the site and scrape the desired information from each corresponding webpage. We will be using the Selenium package for python.

#### There are some challenges to overcome, such as dynamically loaded javascript information, which may not load on the first try, and the website is in Swedish! Google translate will help us with the Swedish, so this isn'tmuch of a problem. We begin by importing some packages, and firing up a headless browser to begin the web crawl.

In [5]:
import time
import random
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import ElementNotVisibleException

options = webdriver.ChromeOptions()
options.binary_location = '/usr/bin/google-chrome-stable'
options.add_argument('headless')
driver = webdriver.Chrome(chrome_options=options)

#### Websites often have a text file, 'robots.txt', that outlines certain behaviors in a webcrawler that are not allowed. A polite webcrawler should check this page before proceeding.

In [ ]:
url='https://www.avanza.se/robots.txt'
driver.get(url)


In [ ]:
url = 'https://www.avanza.se/fonder/lista.html'
driver.get(url)

#### After navigating to the first page containing a links for some of the mutual funds, we are now ready to scrape all of the needed links off the page, and then move on to the next one. The following code will end up crawling through over 40 pages and collecting over a thousand links to individual mutual funds.

In [ ]:
List_link=[]
stop=0
while stop==0:
    time.sleep(random.randint(0,5))
    table=driver.find_element_by_css_selector('#contentTable > tbody')
    List_links = table.find_elements_by_class_name("link")
    for link in List_links:
        List_link.append(link.get_attribute('href'))
    button=driver.find_element_by_css_selector('#contentTableWrapper > div.tCenter.pager.marginTop14px > a.nextPage.marginLeft5px.buttonPaneOuterLink')
    try:
        button.click()
    except ElementNotVisibleException:
        stop+=1
driver.quit()

In [2]:
print(List_link)

['https://www.avanza.se/fonder/om-fonden.html/417579/c-worldwide-medical-small---mid-cap-1a', 'https://www.avanza.se/fonder/om-fonden.html/535295/c-worldwide-biotechnology-1a', 'https://www.avanza.se/fonder/om-fonden.html/2821/bgf-world-mining-a2', 'https://www.avanza.se/fonder/om-fonden.html/260250/handelsbanken-brasilienfond', 'https://www.avanza.se/fonder/om-fonden.html/471/ubs-lux-ef-biotech-usd-p-acc', 'https://www.avanza.se/fonder/om-fonden.html/276448/rhenman-healthcare-equity-l-s-rc1-sek', 'https://www.avanza.se/fonder/om-fonden.html/459/pictet-biotech-r-usd', 'https://www.avanza.se/fonder/om-fonden.html/455/franklin-biotechnology-discv-aaccusd', 'https://www.avanza.se/fonder/om-fonden.html/281971/alfa-axiom-fund', 'https://www.avanza.se/fonder/om-fonden.html/32347/hsbc-gif-turkey-equity-ac', 'https://www.avanza.se/fonder/om-fonden.html/3061/hsbc-gif-chinese-equity-ac', 'https://www.avanza.se/fonder/om-fonden.html/638115/gam-multistock-health-innovt-eq-usd-b', 'https://www.avan

In [1]:
import pickle

List_link=pickle.dump( open( "links.p", "wb" ) )

#List_link=pickle.load( open( "links.p", "rb" ) )

#### The desired links have been stored in a list, and now we are ready to visit each link and scrape the desired fund info. The following function does exactly this given a list of links.

In [13]:
def scraper(list_of_links):
    options = webdriver.ChromeOptions()
    options.binary_location = '/usr/bin/google-chrome-stable'
    options.add_argument('headless')
    driver = webdriver.Chrome(chrome_options=options)
    funds_dict={}
    for link in list_of_links:
        url=link
        driver.get(url)
        time.sleep(random.randint(2,5))
        text = driver.find_element_by_tag_name("body").text
        text=text.split('\n')
        k=0;
        fund_dict={}
        fund_dict['link']=link
        dummy=0
        dummy2=0
        for line in text:
            if line=='Kundservice':
                fund=text[k+1]
                print(fund)
            if line=='Kontaktuppgifter':
                bank=text[k+1]
                fund_dict['bank']=bank
            if line=='Fondinformation':
                info=text[k+1]
                fund_dict['info']=info
            if line=='Typ':
                fund_type=text[k+1]
                fund_dict['fund_type']=fund_type
            if line=='Kategori':
                category=text[k+1]
                fund_dict['category']=category
            if line=='Underkategori':
                sub_categories=text[k+1]
                fund_dict['sub_categories']=sub_categories
            if line=='ISIN':
                isin=text[k+1]
                fund_dict['ISIN']=isin
            if line=='Fondens startdatum':
                start_date=text[k+1]
                fund_dict['start_date']=start_date
            if line=='Morningstar rating' and dummy==1:
                mrnstr=text[k+1]
                if len(mrnstr)>1:
                    mrnstr=int(mrnstr[mrnstr.find('(')+1])
                    fund_dict['morningstar']=mrnstr
            elif line=='Morningstar rating':
                dummy+=1
            if line=='Risk':
                risk=text[k+1]
                if len(risk)>1:
                    risk=int(risk[risk.find('(')+1])
                    fund_dict['risk']=risk
            if line=='Standardavvikelse':
                std_dev=text[k+1]
                std_dev=std_dev.replace(' %','')
                std_dev=float(std_dev.replace(',','.'))
                if std_dev>0:
                    fund_dict['std_dev']=std_dev
            if line=='Sharpe ratio':
                sharpe=text[k+1]
                sharpe=float(sharpe.replace(',','.'))
                if sharpe>0:
                    fund_dict['sharpe_ratio']=sharpe
            if line=='Förvaltningsavgift':
                man_fee=text[k+1]
                man_fee=man_fee.replace(' %','')
                man_fee=float(man_fee.replace(',','.'))
                fund_dict['man_fee']=man_fee
            if line=='Normanbelopp':
                norman=text[k+1]
                norman=norman[:-4]
                norman=int(norman.replace(' ',''))
                fund_dict['normanbelopp']=norman
            if line=='Minsta köpbelopp':
                min_amt=text[k+1]
                min_amt=min_amt[:-4]
                min_amt=int(min_amt.replace(' ',''))
                fund_dict['min_amt']=min_amt
            if line.startswith('Ett år'):
                yr_1=line.split()[2]
                yr_1=yr_1.replace(',','.')
                if len(yr_1)>1:
                    fund_dict['1_yr']=float(yr_1)
                dummy2+=1
            if line.startswith('Tre år') and dummy2>0:
                yr_3=line.split()[2]
                yr_3=yr_3.replace(',','.')
                if len(yr_3)>1:
                    fund_dict['3_yr']=float(yr_3)
            if line.startswith('Fem år'):
                yr_5=line.split()[2]
                yr_5=yr_5.replace(',','.')
                if len(yr_5)>1:
                    fund_dict['5_yr']=float(yr_5)
            if line.startswith('Tio år'):
                yr_10=line.split()[2]
                yr_10=yr_10.replace(',','.')
                if len(yr_10)>1:
                    fund_dict['10_yr']=float(yr_10)
            if line=='Regionsfördelning':
                j=-1;
                regions={}
                while text[k+j]!='Fondens innehav & fördelning':
                    region=text[k+j].rsplit(' ',1)
                    regions[region[0]]=region[1].replace('%','').replace(',','.')
                    j+=-1
                fund_dict['regions']=regions
            if line=='Branschfördelning':
                j=-1;
                industries={}
                while text[k+j]!='Regionsfördelning':
                    industry=text[k+j].rsplit(' ',1)
                    industries[industry[0]]=industry[1].replace('%','').replace(',','.')
                    j+=-1
                fund_dict['industries']=industries
            if line=='Största innehav':
                j=-1;
                holdings={}
                while text[k+j]!='Branschfördelning':
                    holding=text[k+j].rsplit(' ',1)
                    holdings[holding[0]]=holding[1].replace('%','').replace(',','.')
                    j+=-1
                fund_dict['holdings']=holdings
            k+=1;
        funds_dict[fund]=fund_dict
    driver.quit()
    return funds_dict

#### The above function has built in sleep periods, so that we are polite web crawlers and the host site will not be mad at us :). There is important information such as geographical distribution of investments, which is contained in javascript that is not always correctly loaded by the web crawler. Waiting for the page to load does not work in this case. To deal with this we may need to return to the page of a fund that has not loaded properly, so we will use a while loop to keep returnin (after a rest period) until all of the desired information has been collected.

#### (NOTE: Any website can abruptly change, always double check to see that the web scraping code is still behaving properly.

In [14]:
non_geo_links=List_link
while len(non_geo_links)>0:
    dict_of_funds={}
    funds_dict=scraper(non_geo_links)
    geo_counter=0
    non_geo_links=[]
    for fund, fund_dict in funds_dict.items():
        if fund_dict.get('regions')==None:
            non_geo_links.append(fund_dict['link'])
        else:
            dict_of_funds[fund]=fund_dict
            geo_counter+=1
    print(geo_counter)
    print(len(non_geo_links))
    time.sleep(random.randint(60,120))

C WorldWide Medical Small & Mid Cap 1A
C WorldWide Biotechnology 1A
BGF World Mining A2
Handelsbanken Brasilienfond
UBS (Lux) EF Biotech (USD) P-acc
Rhenman Healthcare Equity L/S RC1 SEK
Pictet-Biotech R USD
Franklin Biotechnology Discv A(acc)USD
Alfa Axiom Fund
HSBC GIF Turkey Equity AC
HSBC GIF Chinese Equity AC
GAM Multistock Health Innovt Eq USD B
INVL Baltic Fund
Belfius Eqs China C Cap
Handelsbanken Norge
Allianz Global Metals and Mining A EUR
Mirae Asset China Sector Leader Eq A USD
Danske Invest Latinamerika KL SEK
Fidelity Latin America A-Dis-USD
JPM China A (dist) USD
Lancelot Avalon
C WorldWide Asia 1A
Pictet-Greater China R USD
Handelsbanken Latinamerikafond
Turkisfund Equities A
UBS (Lux) ES Brazil (USD) P-acc
C WorldWide Emerging Markets 1A
Parvest Equity China C C
MS INVF Asia Opportunity A
Pictet - Emerging Europe R EUR
JPM Brazil Equity A (acc) USD
Templeton BRIC A(acc)USD
HSBC GIF Brazil Equity AC
Threadneedle (Lux) Greater China Eqs AU
Alfred Berg Gambak
BGF Latin Am

PriorNilsson Realinvest A
Pictet-Emerging Markets R USD
Excalibur
HSBC GIF Global Em Mkts Equity AC
Placerum Optimera A
PriorNilsson Yield
Lannebo Likviditet
Evli Euro Liquidity B SEK
Simplicity Likviditet
SEB Företagsobligationsfond Flexibel SEK
Aktie-Ansvar Avkastningsfond
Invesco India Equity A USD AD
Lynx
MS INVF Short Maturity Euro Bond AH SEK
Origo Quest 1 C
Franklin K2 Alt Strats A(acc)SEK-H1
Aktie-Ansvar Multistrategi
Danske Invest Sverige Kort Ränta
Fidelity Emerging Asia A-Acc-USD
Invesco Balanced-Risk Allc A SEKH Acc
Aktie-Ansvar Trendhedge
Plain Capital StyX
Alfred Berg Fastighetsfond Norden A
Handelsbanken Hållbar Energi
SEB Commodity Index R SEK
Öhman Företagsobligationsfond Hållbar A
BSF Dynamic Diversified Growth A2 SEK H
Carnegie Likviditetsfond A
SPP Grön Obligationsfond A
Handelsbanken Företagsobligation
SKAGEN Credit SEK
SKAGEN Krona
Nordea Stratega Ränta
Norron Premium RC
SKAGEN m2 A
Swedbank Robur Penningmarknadsfond
Gustavia Ryssland
Handelsbanken Kortränta Crite

Handelsbanken Nordenfond
Jupiter Global Value L USD Acc
Handelsbanken Nordiska Småbolag
Pictet-Emerg Mkt Sustainable Eq P USD
Swedbank Robur Småbolagsfond Norden
UB Infra A
Candriam Eqs L Australia C AUD Cap
Fondita Nordic Micro Cap B
DNB Grönt Norden
Ålandsbanken Norden Aktie EUR
Amundi Fds Eq MENA AU-C
Solidar SmartBeta Trend
Handelsbanken Euro Ränta
Swedbank Robur Access Mix
Pictet-Health R USD
SEB Europafond Småbolag
UBS (Lux) ES Emerging Mkts Sust (USD) P
Öhman Etisk Index Pacific
Henderson Horizon PanEurpPtyEqs A2EURAcc
Ålandsbanken Norden Aktie SEK
BGF Continental European Flex A2RF EUR
Danske Invest SICAV Global EM A
Swedbank Robur Småbolagsfond Europa
Fidelity Euro Cash A-Dis-EUR
Danske Invest Aktiv Förmögenhetsförv
SEB Korträntefond EUR LUX C
Parvest Eq BestSelAsia ex-Jpn C C
Swedbank Robur Bas Mix
FIM Sahara A
Fondita Nordic Small Cap B
Parvest Equity Wld Emerg Low Vol C C
Ålandsbanken Cash Manager B
Swedbank Robur Technology
Länsförsäkringar Trygghetsfond A
BGF Japan Small 

SPP Global Topp 100
Handelsbanken Sverigefond
HSBC GIF Global Em Mkts Local Dbt AC
Threadneedle (Lux) US Contr Cr Eqs AU
Pictet-Indian Equities R USD
CS (Lux) Small&Mid Cap Germany Eq B EUR
SEB Hållbarhetsfond Global
Evli Aktieindex B
Navigera Global Change
Pictet - Robotics P EUR
Öhman Hjärt-Lungfond
Fair Play
Handelsbanken Amerika Småbolag (A USD)
Templeton Global Balanced A(acc)USD
Didner & Gerge Aktiefond
Baring Europa A USD Inc
Parworld Quant Eqty World Guru C C
Fidelity European Smlr Coms A-Dis-EUR
Aberdeen Global Select Em Mkts Bd A2 USD
Fidelity American Growth A-Dis-USD
FIM Balanserad
SEB Swedish Value Fund
SEB Nordamerikafond
C WorldWide Stable Equities 1A
Fidelity Global Corporate Bd A-Acc-EUR
Monyx Strategi Offensiv DC
Nordea Småbolagsfond Sverige
DNB Private Equity retail B
Evli Sverige Select B
Humle Kapitalförvaltningsfond
Länsförsäkringar Sverige Aktiv A
Pictet-Emerging Local Ccy Dbt R USD
SEB Europafond
Swedbank Robur Access Sverige
Threadneedle (Lux) American AU
Navig

Baring Global Leaders A USD Inc
Öhman Etisk Index Europa
HSBC GIF Global Emerg Mkts Bd AC
PriorNilsson Smart Global
CB European Quality Fund A
Danske Invest USA KL SEK
Gustavia Sverige
Fidelity Malaysia A-Dis-USD
DNB Fund Global SRI retail A
Fidelity Euro Blue Chip A-Dis-EUR
Fidelity Japan A-Dis-JPY
Sparinvest SICAV European Value EUR R
JPM US Select Equity A (acc) USD
Aberdeen Global World Equity A2 USD
Parvest Equity Indonesia C C
Swedbank Robur Småbolagsfond Sverige
Allianz Europe Small Cap Equity AT EUR
Templeton Growth (Euro) A(acc)EUR
Templeton European A(acc)EUR
JPM Japan Select Equity A (acc) JPY
Threadneedle (Lux) Global Technology AU
Allianz Europe Equity Gr Sel AT EUR
Pictet-Japanese Equity Opps R JPY
Stewart Investors Wldwd Sustbl A Acc GBP
Danske Invest SICAV Gl Stockpicking A
Länsförsäkringar Europa Aktiv A
HSBC GIF Asian Currencies Bond AC
Fidelity Asian High Yield A-Acc-EUR
Fidelity US High Yield A-Acc-EUR
LO Funds Europe High Convc EUR PA
Candriam Sustainable Europe C 

...

CS(Lux) Glbl Prpty Ttl Ret Eq BH EUR Acc
Templeton China A(acc)USD
Nordea Indexfond Global
RobecoSAM Global Small Cap Eqs B EUR
Fidelity Indonesia A-Dis-USD
SEB Asset Selection Def C H-SEK
SPP Penningmarknadsfond
Parvest Equity USA Small Cap C C
Atlant Stability
UBS (Lux) EF Mid Caps USA (USD) P
BSF Fixed Income Strategies A2 EUR
Allianz Global High Yield A USD
Atlant Multi Strategy External
Templeton Emerging Mkts Bd A(Qdis)USD
FIM Emerging markets ESG
Invesco Asia Infrastructure A USD Acc
Threadneedle (Lux) Glbl Mlt Ast Inc AU
LO Funds Golden Age USD PA
Handelsbanken Hållbar Energi
Sparinvest SICAV Eth HY Value Bds EUR R
Danske Invest SICAV Eur LS Dyn A SEK Hp
Catella Nordic L/S Equity R
Delphi Nordic
Spiltan Högräntefond
Ålandsbanken Premium 30 SEK
Danske Invest Sverige Kort Ränta
Quesada Balanserad A
Nordea Stabila Aktier H-SEK
Coeli Multistrategi
Lannebo Corporate Bond A
Invesco Asia Consumer Demand A USD Acc
Templeton Emerging Markets A(acc)USD
Öhman Etisk Index Pacific
Nordea In

Aberdeen Global II USD Bond A2 USD
PriorNilsson Sverige Aktiv A
East Capital Emerging Asia A SEK
HSBC GIF Euroland Eq SmallerCompanies AC
Schroder ISF Glbl Demo Opps AUSD
Monyx Strategi Världen DC
Quesada Global
Fidelity Nordic A-Dis-SEK
DNB USA
Templeton BRIC A(acc)USD
BMO Global Eq Mkt Netrl (SICAV) A SEK H
Amundi Fds CPR Global Resources AU-C
Henderson Horizon Pan Eurp Eq A2 EUR Acc
First State Glbl Prpty Secs A Acc GBP
JPM Africa Equity A perf (acc) USD
Fidelity Pacific A-Dis-USD
Allianz Income and Growth AT USD
MS INVF Indian Equity A
JPM Global Strat Bd A perf (acc) SEKH
SPP Företagsobligationsfond A
SEB Läkemedelsfond
Avanza 75
Fidelity Asia Focus A-Dis-USD
Henderson Horizon Japan Opps A2 USD Acc
BGF World Gold A2RF
IKC Global Healthcare A
SEB Globalfond Chans/Risk Lux
Tellus Investmentbolag
Länsförsäkringar Multistrategi
Aviva Investors Emerging Eurp Eq B EUR Acc
Fidelity China RMB Bond A-Acc-RMB
BMO F&C Diversified Growth A SEK Hedged
JPM Asia Pacific Equity A (dist) USD
Hende

Fidelity Asia Pacific Div A-Dis-USD
Henderson Gartmore PanEurp SmrComs REURAcc
Fidelity Indonesia A-Dis-USD
Templeton Korea A(acc)USD
SPP Penningmarknadsfond
Parvest Equity USA Small Cap C C
Atlant Stability
UBS (Lux) EF Mid Caps USA (USD) P
Pictet-Emerging Markets R USD
Amundi Fds Eq Global Conservative AU-C
Henderson Horizon Jpns Smr Coms A2USD Acc
JPM Europe Strategic Growth A (dist) EUR
Pictet-Water R EUR
Allianz Global High Yield A USD
Atlant Multi Strategy External
Schroder ISF QEP Glbl Act Val A Acc
Templeton Emerging Mkts Bd A(Qdis)USD
FIM Emerging markets ESG
Invesco Asia Infrastructure A USD Acc
Parworld Quant Eqty World Guru C C
Allianz Global Small Cap Equity AT USD
RobecoSAM Sustainable Water B EUR
Spiltan Aktiefond Småland
SEB Nordenfond - Lux
Länsförsäkringar Småbolag Sverige A
LO Funds Golden Age USD PA
Handelsbanken Hållbar Energi
Swedbank Robur Global Emerging Markets
Baring ASEAN Frontiers A USD Inc
Fidelity Asia Focus A-Dis-USD
Sparinvest SICAV Eth HY Value Bds EUR 

Öhman Etisk Index Europa
Catella Sverige Index A
Simplicity Europa
BGF Asian Growth Leaders A2RF USD
Henderson Gartmore PanEurp SmrComs REURAcc
Allianz Global Small Cap Equity AT USD
Swedbank Robur Sweden High Dividend
Allianz Thailand Equity A EUR
Pictet-Emerg Mkt Sustainable Eq P USD
Spiltan Högräntefond
Amundi Fds CPR Global Gold Mines AU-C
CS (Lux) European Div Plus Eq B EUR
RobecoSAM Sustainable Water B EUR
Swedbank Robur Globalfond
Cliens Räntefond Kort
Handelsbanken Europa Selektiv
Simplicity Småbolag Sverige A
IKC Global Infrastructure A
Länsförsäkringar Sverige Aktiv A
Nordea 1 - Global Dividend BP SEK
Stewart Investors Worldwide Eq A Acc GBP
GAM Star (Lux) - European Momentum D EUR
Tundra Sustainable Frontier A SEK
Evli Aktieindex B
C WorldWide Asia 1A
Coeli Global Selektiv R SEK
SEB Nordamerikafond
AMF Räntefond Kort
SPP Aktiefond Japan
Länsförsäkringar Småbolag Sverige A
Cicero China Index A
Lannebo Mixfond
SEB Asienfond ex Japan - Lux
Schroder ISF Global Energy A
Baring AS

KeyboardInterrupt: 

#### We will store our python dictionary (aka hash table) as a json, so it can be easily used with other languages.

In [ ]:
import json
 
jsin = json.dumps(dict_of_funds)
f = open("dict_of_funds.json","w")
f.write(jsin)
f.close()

#### That's it! We are done with the data collection stage, and we can now move on to some exciting analysis using text mining and NLP techniques.